## Importing Libraries











































































































































































In [1]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from mlflow.models import infer_signature
from pprint import pprint
from typing import Union, Dict, List, Tuple
import numpy as np
import pandas as pd
import yaml
import boto3
import joblib
import os
import math
import mlflow
import optuna
import warnings
import shutil

warnings.filterwarnings("ignore")

In [2]:
def read_yaml_file(path, file):
    # reading credentials files
    with open(f"{os.path.join(path, file)}") as f:
        try:
            content = yaml.safe_load(f)
        except yaml.YAMLError as e:
            raise e

    return content


CONFIG_PATH = os.path.join("src", "config")

In [3]:
credentials_config = read_yaml_file(path=CONFIG_PATH, file="credentials.yaml")

general_settings = read_yaml_file(path=CONFIG_PATH, file="settings.yaml")

if credentials_config["EC2"] != "YOUR_EC2_INSTANCE_URL":
    mlflow.set_tracking_uri(f"http://{credentials_config['EC2']}:5000")
else:
    mlflow.set_tracking_uri(f"http://mlflow:5000")

print(f"Tracking Server URI: '{mlflow.get_tracking_uri()}'")

SEED = 42
ARTIFACTS_OUTPUT_PATH = general_settings["ARTIFACTS_PATH"]
FEATURES_OUTPUT_PATH = general_settings["FEATURES_PATH"]
RAW_FILE_PATH = os.path.join(
    general_settings["DATA_PATH"], general_settings["RAW_FILE_NAME"]
)
PROCESSED_RAW_FILE = "Preprocessed_" + general_settings["RAW_FILE_NAME"]
PROCESSED_RAW_FILE_PATH = os.path.join(
    general_settings["DATA_PATH"], PROCESSED_RAW_FILE
)
FEATURE_SELECTION_EXPERIMENT_NAME = "feature-selection-experimentation"
HYPERPARAMETER_TUNING_EXPERIMENT_NAME = "hyperparameters-tuning-experimentation"

Tracking Server URI: 'http://mlflow:5000'


## Loading Essentials

In [4]:
# downloading the preprocessed dataset from the aws s3 bucket
if credentials_config["S3"] != "YOUR_S3_BUCKET_URL":
    # configuring AWS credentials
    os.environ["AWS_ACCESS_KEY_ID"] = credentials_config["AWS_ACCESS_KEY"]
    os.environ["AWS_SECRET_ACCESS_KEY"] = credentials_config["AWS_SECRET_KEY"]

    # downloading preprocessed dataset
    s3 = boto3.client(
        "s3",
        aws_access_key_id=credentials_config["AWS_ACCESS_KEY"],
        aws_secret_access_key=credentials_config["AWS_SECRET_KEY"]
    )
    s3.download_file(
        credentials_config["S3"],
        PROCESSED_RAW_FILE,
        PROCESSED_RAW_FILE_PATH
    )

    # downloading artifacts from the aws s3 bucket
    !aws s3 cp --recursive s3://{credentials_config["S3"]}/artifacts {ARTIFACTS_OUTPUT_PATH}

    # downloading models from the aws s3 bucket
    !aws s3 cp --recursive s3://{credentials_config["S3"]}/features {FEATURES_OUTPUT_PATH}

In [5]:
# loading features
X_train = joblib.load(os.path.join(FEATURES_OUTPUT_PATH, "X_train.pkl"))
y_train = joblib.load(os.path.join(FEATURES_OUTPUT_PATH, "y_train.pkl"))

X_valid = joblib.load(os.path.join(FEATURES_OUTPUT_PATH, "X_valid.pkl"))
y_valid = joblib.load(os.path.join(FEATURES_OUTPUT_PATH, "y_valid.pkl"))

# loading artifacts
sc = joblib.load(os.path.join(ARTIFACTS_OUTPUT_PATH, "features_sc.pkl"))
ohe = joblib.load(os.path.join(ARTIFACTS_OUTPUT_PATH, "features_ohe.pkl"))
ohe_label = joblib.load(os.path.join(ARTIFACTS_OUTPUT_PATH, "label_ohe.pkl"))

# loading feature columns
temp_df = pd.read_csv(PROCESSED_RAW_FILE_PATH, sep=",")
FEATURES_NAME = temp_df.columns.tolist()
del temp_df

# REMOVE THIS CELL IN ACTUAL EXECUTIONS
this is to make execution faster

In [6]:
X_train = X_train[:1000, :10]
y_train = y_train[:1000]
X_valid = X_valid[:200, :10]
y_valid = y_valid[:200]

## Feature Selection Experimentation

In [7]:
# creating the baseline models
dt = DecisionTreeClassifier(random_state=SEED)
rf = RandomForestClassifier(random_state=SEED, verbose=0, n_jobs=-1)
xg = XGBClassifier(random_state=SEED, n_jobs=-1)
lg = LGBMClassifier(random_state=SEED, verbose=-1, objective="multiclass")
cb = CatBoostClassifier(random_seed=SEED, verbose=0, allow_writing_files=False)

In [8]:
def apply_feature_selection(
    model: Union[
        DecisionTreeClassifier,
        RandomForestClassifier,
        XGBClassifier,
        LGBMClassifier,
        CatBoostClassifier,
    ],
    number_features: int,
    X_train: np.ndarray,
    y_train: np.array,
    X_valid: np.ndarray,
    y_valid: np.array,
) -> Dict:
    # initializing and fitting the sfs class
    sfs = SequentialFeatureSelector(model, n_features_to_select=number_features, cv=3)
    sfs.fit(X=X_train, y=y_train)

    # getting the indexes of the best features
    selected_features_indexes = np.argwhere(sfs.get_support()).reshape(-1)

    reduced_X_train = sfs.transform(X_train)
    reduced_X_valid = sfs.transform(X_valid)

    # training the model
    model.fit(reduced_X_train, y_train)

    # calculating the training f1 score
    predicted_y_train = model.predict(reduced_X_train)
    train_f1 = f1_score(y_true=y_train, y_pred=predicted_y_train, average="weighted")

    # calculating the validation f1 score
    predicted_y_valid = model.predict(reduced_X_valid)
    valid_f1 = f1_score(y_true=y_valid, y_pred=predicted_y_valid, average="weighted")

    # inferring the signature of the trained model
    signature = infer_signature(
        model_input=reduced_X_train, model_output=predicted_y_train
    )

    # saving the metrics and artifacts that we want to log in mlflow
    selected_features_names = list(
        map(lambda i: FEATURES_NAME[i], selected_features_indexes.tolist())
    )

    results = {
        "train_f1": train_f1,
        "valid_f1": valid_f1,
        "features": selected_features_names,
        "model": model,
        "model_signature": signature,
    }

    return results


def set_configurations_mlflow(
    model: Union[
        DecisionTreeClassifier,
        RandomForestClassifier,
        XGBClassifier,
        LGBMClassifier,
        CatBoostClassifier,
    ],
    y_train: np.array,
    y_valid: np.array,
) -> Tuple[np.array, np.array, str, str]:
    # reshaping the target values (if needed) and setting the run name and which
    # flavor is being used for each machine learning model
    if isinstance(model, DecisionTreeClassifier):
        y_train = np.argmax(y_train, axis=1)
        y_valid = np.argmax(y_valid, axis=1)
        run_name = "decision_tree"
        flavor = "sklearn"

    if isinstance(model, RandomForestClassifier):
        run_name = "random_forest"
        flavor = "sklearn"

    if isinstance(model, XGBClassifier):
        run_name = "xgboost"
        flavor = "xgboost"

    if isinstance(model, LGBMClassifier):
        y_train = np.argmax(y_train, axis=1)
        y_valid = np.argmax(y_valid, axis=1)
        run_name = "lightgbm"
        flavor = "lightgbm"

    if isinstance(model, CatBoostClassifier):
        y_train = np.argmax(y_train, axis=1)
        y_valid = np.argmax(y_valid, axis=1)
        run_name = "catboost"
        flavor = "catboost"

    # disabling some options of the current flavor's autolog
    if flavor == "sklearn":
        mlflow.sklearn.autolog(
            log_models=False,
            log_post_training_metrics=False,
            log_model_signatures=False,
            log_input_examples=True,
            log_datasets=False,
            silent=True,
            disable=True,
        )
    elif flavor == "xgboost":
        mlflow.xgboost.autolog(
            log_models=False,
            log_model_signatures=False,
            log_input_examples=True,
            log_datasets=False,
            silent=True,
            disable=True,
        )
    elif flavor == "lightgbm":
        mlflow.lightgbm.autolog(
            log_models=False,
            log_model_signatures=False,
            log_input_examples=True,
            log_datasets=False,
            silent=True,
            disable=True,
        )
    elif flavor == "catboost":
        # there is no autolog implemented for catboost
        pass

    return y_train, y_valid, run_name, flavor


def run_feature_selection_experiment(
    models: List, min_features: int, max_features: int, experiment_id: str
) -> None:
    for model in models:
        # reshaping the target values (if needed) and setting some mlflow's configuration
        new_y_train, new_y_valid, run_name, flavor = set_configurations_mlflow(
            model=model, y_train=y_train, y_valid=y_valid
        )

        # starting a new run for the current model
        with mlflow.start_run(experiment_id=experiment_id, run_name=run_name):
            pprint(f"Starting the run for the {run_name} model!\n")

            for i, n_features in enumerate(range(min_features, max_features + 1)):
                # creating a nested run inside the model's main run
                with mlflow.start_run(
                    experiment_id=experiment_id,
                    run_name=f"{run_name}_experiment_{i}",
                    nested=True,
                ):
                    # running the feature selection main function
                    results = apply_feature_selection(
                        model=model,
                        number_features=n_features,
                        X_train=X_train,
                        y_train=new_y_train,
                        X_valid=X_valid,
                        y_valid=new_y_valid,
                    )

                    # logging the trained model
                    if flavor == "sklearn":
                        mlflow.sklearn.log_model(
                            results["model"],
                            run_name,
                            signature=results["model_signature"],
                        )
                        # logging the model"s default parameters
                        mlflow.log_params(results["model"].get_params(deep=True))
                    elif flavor == "xgboost":
                        mlflow.xgboost.log_model(
                            results["model"],
                            run_name,
                            signature=results["model_signature"],
                        )
                        # logging the model's default parameters
                        mlflow.log_params(results["model"].get_params(deep=True))
                    elif flavor == "lightgbm":
                        mlflow.lightgbm.log_model(
                            results["model"],
                            run_name,
                            signature=results["model_signature"],
                        )
                        # logging the model's default parameters
                        mlflow.log_params(results["model"].get_params())
                    elif flavor == "catboost":
                        mlflow.catboost.log_model(
                            results["model"],
                            run_name,
                            signature=results["model_signature"],
                        )
                        # logging the model's default parameters
                        mlflow.log_params(results["model"].get_all_params())

                    # logging the training and validation scores
                    mlflow.log_metric("train_f1", results["train_f1"])
                    mlflow.log_metric("valid_f1", results["valid_f1"])

                    # logging the artifacts (original dataset, features, and encoders objects)
                    mlflow.log_artifact(PROCESSED_RAW_FILE_PATH)
                    mlflow.log_artifact(ARTIFACTS_OUTPUT_PATH)
                    mlflow.log_artifact(FEATURES_OUTPUT_PATH)

                    # logging the indexes of the best features
                    mlflow.log_param("features", results["features"])

In [9]:
models = [dt, rf, xg, lg, cb]
min_features = math.floor(X_train.shape[1] * 0.2)
max_features = math.floor(X_train.shape[1] * 0.5)

# creating a new mlflow's experiment
experiment_id = mlflow.create_experiment(
    name=FEATURE_SELECTION_EXPERIMENT_NAME,
)

# running the feature selection experiments
run_feature_selection_experiment(
    models=models,
    min_features=min_features,
    max_features=max_features,
    experiment_id=experiment_id,
)

'Starting the run for the decision_tree model!\n'


2024/12/12 20:13:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run decision_tree_experiment_0 at: http://mlflow:5000/#/experiments/979374007911968531/runs/5a07a9d4276b4a858272e68987d609c9.
2024/12/12 20:13:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:13:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run decision_tree_experiment_1 at: http://mlflow:5000/#/experiments/979374007911968531/runs/8e2e6f8b1a194404bd9b7c3c9d005efb.
2024/12/12 20:13:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:13:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run decision_tree_experiment_2 at: http://mlflow:5000/#/experiments/979374007911968531/runs/a2f469ad672c4963b58f52724e4eda6b.
2024/12/12 20:13:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experi

'Starting the run for the random_forest model!\n'


2024/12/12 20:14:36 INFO mlflow.tracking._tracking_service.client: 🏃 View run random_forest_experiment_0 at: http://mlflow:5000/#/experiments/979374007911968531/runs/fdb6a1b5c5d646d0902bb5b3688f9823.
2024/12/12 20:14:36 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:15:28 INFO mlflow.tracking._tracking_service.client: 🏃 View run random_forest_experiment_1 at: http://mlflow:5000/#/experiments/979374007911968531/runs/55374abcd8a34c18a6edb8cc5f05b024.
2024/12/12 20:15:28 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:16:34 INFO mlflow.tracking._tracking_service.client: 🏃 View run random_forest_experiment_2 at: http://mlflow:5000/#/experiments/979374007911968531/runs/ac4138d846064879abf6df45dbf738c6.
2024/12/12 20:16:34 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experi

'Starting the run for the xgboost model!\n'


2024/12/12 20:18:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run xgboost_experiment_0 at: http://mlflow:5000/#/experiments/979374007911968531/runs/7d9bc3a719244e81945a496903f43a8b.
2024/12/12 20:18:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:18:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run xgboost_experiment_1 at: http://mlflow:5000/#/experiments/979374007911968531/runs/81d46dcd82484ac995ff7fb2a7db0169.
2024/12/12 20:18:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:18:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run xgboost_experiment_2 at: http://mlflow:5000/#/experiments/979374007911968531/runs/e3e8323750fa43a3a7001a5aa1fab6c4.
2024/12/12 20:18:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911

'Starting the run for the lightgbm model!\n'


2024/12/12 20:19:09 INFO mlflow.tracking._tracking_service.client: 🏃 View run lightgbm_experiment_0 at: http://mlflow:5000/#/experiments/979374007911968531/runs/ab1dce7e0e244adba4c1dde50d6d1126.
2024/12/12 20:19:09 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:19:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run lightgbm_experiment_1 at: http://mlflow:5000/#/experiments/979374007911968531/runs/04d910d2739e41e78b03befc43162464.
2024/12/12 20:19:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:19:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run lightgbm_experiment_2 at: http://mlflow:5000/#/experiments/979374007911968531/runs/7a501f7ddadd4105a3630255acc0fd23.
2024/12/12 20:19:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007

'Starting the run for the catboost model!\n'


2024/12/12 20:20:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run catboost_experiment_0 at: http://mlflow:5000/#/experiments/979374007911968531/runs/43a69b97ad134a139954f62477821176.
2024/12/12 20:20:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:20:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run catboost_experiment_1 at: http://mlflow:5000/#/experiments/979374007911968531/runs/f69ff4862bc04892ab2cc28d9eaf1aec.
2024/12/12 20:20:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007911968531.
2024/12/12 20:21:44 INFO mlflow.tracking._tracking_service.client: 🏃 View run catboost_experiment_2 at: http://mlflow:5000/#/experiments/979374007911968531/runs/187fd522f7724304b2f4648c22b9d421.
2024/12/12 20:21:44 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/979374007

## Hyperparameters Tuning

In [10]:
class Objective:
    def __init__(
        self,
        run_name: str,
        experiment_id: str,
        X_train: np.ndarray,
        y_train: np.array,
        X_valid: np.ndarray,
        y_valid: np.array,
        indexes: List,
    ) -> None:
        self.run_name = run_name
        self.experiment_id = experiment_id
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.indexes_name = indexes
        self.indexes = [FEATURES_NAME.index(i) for i in indexes]

        if self.run_name in ["decision_tree", "lightgbm", "catboost"]:
            self.y_train = np.argmax(self.y_train, axis=1)
            self.y_valid = np.argmax(self.y_valid, axis=1)

        self.X_train = self.X_train[:, self.indexes]
        self.X_valid = self.X_valid[:, self.indexes]

    def __call__(self, trial: optuna.trial.Trial) -> float:
        with mlflow.start_run(experiment_id=self.experiment_id, nested=True):
            if self.run_name == "decision_tree":
                params = {
                    "max_depth": trial.suggest_int("max_depth", 2, 32, step=2),
                    "min_samples_split": trial.suggest_int(
                        "min_samples_split", 2, 8, step=1
                    ),
                    "min_samples_leaf": trial.suggest_int(
                        "min_samples_leaf", 1, 6, step=1
                    ),
                    "min_weight_fraction_leaf": trial.suggest_float(
                        "min_weight_fraction_leaf", 0, 0.5, step=0.1
                    ),
                    "max_leaf_nodes": trial.suggest_int(
                        "max_leaf_nodes", 2, 16, step=2
                    ),
                    "random_state": SEED,
                }
                model = DecisionTreeClassifier(**params)

            if self.run_name == "random_forest":
                params = {
                    "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
                    "max_depth": trial.suggest_int("max_depth", 10, 50),
                    "min_samples_split": trial.suggest_int("min_samples_split", 2, 32),
                    "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 32),
                    "random_state": SEED,
                    "n_jobs": -1,
                }
                model = RandomForestClassifier(**params)

            if self.run_name == "xgboost":
                params = {
                    "booster": trial.suggest_categorical(
                        "booster", ["gbtree", "gblinear", "dart"]
                    ),
                    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
                    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
                    "random_state": SEED,
                    "n_jobs": -1,
                }
                model = XGBClassifier(**params)

            if self.run_name == "lightgbm":
                params = {
                    "objective": "multiclass",
                    "verbosity": -1,
                    "random_state": SEED,
                    "n_jobs": -1,
                    "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
                    "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
                    "num_leaves": trial.suggest_int("num_leaves", 2, 256),
                    "feature_fraction": trial.suggest_float(
                        "feature_fraction", 0.4, 1.0
                    ),
                    "bagging_fraction": trial.suggest_float(
                        "bagging_fraction", 0.4, 1.0
                    ),
                    "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
                    "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
                }
                model = LGBMClassifier(**params)

            if self.run_name == "catboost":
                params = {
                    "random_seed": SEED,
                    "verbose": 0,
                    "allow_writing_files": False,
                    "colsample_bylevel": trial.suggest_float(
                        "colsample_bylevel", 0.01, 0.1
                    ),
                    "depth": trial.suggest_int("depth", 1, 12),
                    "boosting_type": trial.suggest_categorical(
                        "boosting_type", ["Ordered", "Plain"]
                    ),
                    "bootstrap_type": trial.suggest_categorical(
                        "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
                    ),
                }
                model = CatBoostClassifier(**params)

            model.fit(X=self.X_train, y=self.y_train)

            # calculating the training f1 score
            train_prediction = model.predict(self.X_train)
            train_f1 = f1_score(
                y_true=self.y_train, y_pred=train_prediction, average="weighted"
            )

            # calculating the validation f1 score
            valid_prediction = model.predict(self.X_valid)
            valid_f1 = f1_score(
                y_true=self.y_valid, y_pred=valid_prediction, average="weighted"
            )

            # logging the training and validation scores
            mlflow.log_metric("train_f1", train_f1)
            mlflow.log_metric("valid_f1", valid_f1)

            # inferring the signature of the trained model
            signature = infer_signature(
                model_input=self.X_train, model_output=train_prediction
            )

            # saving the trained model
            if self.run_name in ["decision_tree", "random_forest"]:
                # sklearn flavor
                mlflow.sklearn.log_model(model, self.run_name, signature=signature)
                # logging the model"s default parameters
                mlflow.log_params(model.get_params(deep=True))
            elif self.run_name == "xgboost":
                mlflow.xgboost.log_model(model, self.run_name, signature=signature)
                # logging the model's default parameters
                mlflow.log_params(model.get_params())
            elif self.run_name == "lightgbm":
                mlflow.lightgbm.log_model(model, self.run_name, signature=signature)
                # logging the model's default parameters
                mlflow.log_params(model.get_params())
            elif self.run_name == "catboost":
                mlflow.catboost.log_model(model, self.run_name, signature=signature)
                # logging the model's default parameters
                mlflow.log_params(model.get_all_params())

        return valid_f1

In [11]:
# creating a new mlflow's experiment
hpt_experiment_id = mlflow.create_experiment(
    name=HYPERPARAMETER_TUNING_EXPERIMENT_NAME,
)

### Decision Tree

In [12]:
dt_run_name = "decision_tree"
dt_features_indexes = ["Gender_x0_Male", "family_history_with_overweight_x0_yes"]

with mlflow.start_run(experiment_id=hpt_experiment_id, run_name=dt_run_name):
    objective = Objective(
        run_name=dt_run_name,
        experiment_id=hpt_experiment_id,
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid,
        indexes=dt_features_indexes,
    )

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

[I 2024-12-12 20:22:57,078] A new study created in memory with name: no-name-d2dadb7e-d3be-43d6-a056-4c98d983be7d
2024/12/12 20:22:59 INFO mlflow.tracking._tracking_service.client: 🏃 View run respected-moose-934 at: http://mlflow:5000/#/experiments/666506561086108314/runs/02a062cefb834f238ab513272bfd8e4a.
2024/12/12 20:22:59 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/666506561086108314.
[I 2024-12-12 20:22:59,129] Trial 0 finished with value: 0.2128795645510754 and parameters: {'max_depth': 14, 'min_samples_split': 2, 'min_samples_leaf': 6, 'min_weight_fraction_leaf': 0.30000000000000004, 'max_leaf_nodes': 6}. Best is trial 0 with value: 0.2128795645510754.
2024/12/12 20:23:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run worried-sheep-537 at: http://mlflow:5000/#/experiments/666506561086108314/runs/82d229835ca7462886221a66d81c3302.
2024/12/12 20:23:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment

#### Random Forest

In [13]:
rf_run_name = "random_forest"
rf_features_indexes = ["Age_x0_q4", "CAEC_x0_Frequently"]

with mlflow.start_run(experiment_id=hpt_experiment_id, run_name=rf_run_name):
    objective = Objective(
        run_name=rf_run_name,
        experiment_id=hpt_experiment_id,
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid,
        indexes=rf_features_indexes,
    )

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

[I 2024-12-12 20:23:18,074] A new study created in memory with name: no-name-e462ef19-c7f5-4d41-b673-8d44b6e33a17
2024/12/12 20:23:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run rebellious-seal-99 at: http://mlflow:5000/#/experiments/666506561086108314/runs/d3e330461f2d4c92be9bac03b30aefe4.
2024/12/12 20:23:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/666506561086108314.
[I 2024-12-12 20:23:23,563] Trial 0 finished with value: 0.07529411764705882 and parameters: {'n_estimators': 422, 'max_depth': 31, 'min_samples_split': 19, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.07529411764705882.
2024/12/12 20:23:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run salty-mare-519 at: http://mlflow:5000/#/experiments/666506561086108314/runs/ef6baeeae0fe4503b2eaffd385809308.
2024/12/12 20:23:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/666506561086

#### XGBoost

In [14]:
xgb_run_name = "xgboost"
xg_features_indexes = ["Age_x0_q4", "CAEC_x0_Frequently"]

with mlflow.start_run(experiment_id=hpt_experiment_id, run_name=xgb_run_name):
    objective = Objective(
        run_name=xgb_run_name,
        experiment_id=hpt_experiment_id,
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid,
        indexes=xg_features_indexes,
    )

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

[I 2024-12-12 20:24:27,743] A new study created in memory with name: no-name-b12b2d6d-0a61-4957-9411-a3fa39db56e6
2024/12/12 20:24:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run abrasive-shrike-90 at: http://mlflow:5000/#/experiments/666506561086108314/runs/d7bc33ebba9647c180f957bc4c800bee.
2024/12/12 20:24:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/666506561086108314.
[I 2024-12-12 20:24:29,875] Trial 0 finished with value: 0.0 and parameters: {'booster': 'gblinear', 'lambda': 0.8654930730948676, 'alpha': 2.55673669708507e-07}. Best is trial 0 with value: 0.0.
2024/12/12 20:24:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run skillful-shrike-21 at: http://mlflow:5000/#/experiments/666506561086108314/runs/40b2763b6b1041a8907ee389edba8982.
2024/12/12 20:24:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/666506561086108314.
[I 2024-12-12 20:24:32

#### LightGBM

In [15]:
lg_run_name = "lightgbm"
lg_features_indexes = ["Gender_x0_Male", "family_history_with_overweight_x0_yes"]

with mlflow.start_run(experiment_id=hpt_experiment_id, run_name=lg_run_name):
    objective = Objective(
        run_name=lg_run_name,
        experiment_id=hpt_experiment_id,
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid,
        indexes=lg_features_indexes,
    )

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

[I 2024-12-12 20:24:57,461] A new study created in memory with name: no-name-12e9e131-43ba-4b9d-9cf9-7c40a43b2e8f
2024/12/12 20:25:00 INFO mlflow.tracking._tracking_service.client: 🏃 View run omniscient-kit-873 at: http://mlflow:5000/#/experiments/666506561086108314/runs/d524016eae7b45bebcbe745b1edaad96.
2024/12/12 20:25:00 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/666506561086108314.
[I 2024-12-12 20:25:00,418] Trial 0 finished with value: 0.36202092481162246 and parameters: {'lambda_l1': 0.0001470581084493255, 'lambda_l2': 1.906710004483921e-07, 'num_leaves': 240, 'feature_fraction': 0.8495755388337749, 'bagging_fraction': 0.5314116688792873, 'bagging_freq': 2, 'min_child_samples': 6}. Best is trial 0 with value: 0.36202092481162246.
2024/12/12 20:25:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run unruly-whale-947 at: http://mlflow:5000/#/experiments/666506561086108314/runs/eb3be66b087f43598bc9faffbc9ab18c.
2024

#### CatBoost

In [16]:
cb_run_name = "catboost"
cb_features_indexes = ["Gender_x0_Male", "family_history_with_overweight_x0_yes"]

with mlflow.start_run(experiment_id=hpt_experiment_id, run_name=cb_run_name):
    objective = Objective(
        run_name=cb_run_name,
        experiment_id=hpt_experiment_id,
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid,
        indexes=cb_features_indexes,
    )

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

[I 2024-12-12 20:25:23,989] A new study created in memory with name: no-name-8b6ae6b5-d361-4d52-baf9-3376af110045
2024/12/12 20:25:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run upset-mole-327 at: http://mlflow:5000/#/experiments/666506561086108314/runs/8206457522fb4ab5b95df0de51235081.
2024/12/12 20:25:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000/#/experiments/666506561086108314.
[I 2024-12-12 20:25:27,949] Trial 0 finished with value: 0.36202092481162246 and parameters: {'colsample_bylevel': 0.03941303027017113, 'depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli'}. Best is trial 0 with value: 0.36202092481162246.
2024/12/12 20:25:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run overjoyed-seal-877 at: http://mlflow:5000/#/experiments/666506561086108314/runs/91c641d231df4f789712dc8a8759aba9.
2024/12/12 20:25:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://mlflow:5000

In [ ]:
if credentials_config["EC2"] != "YOUR_EC2_INSTANCE_URL":
    # removing downloaded dataset from local
    os.remove(PROCESSED_RAW_FILE_PATH)

    # removing the local artifacts and features
    shutil.rmtree(ARTIFACTS_OUTPUT_PATH)
    shutil.rmtree(FEATURES_OUTPUT_PATH)

## Registering Best Models

In [18]:
run_id = "100d86e2471240dbb4b2199533bf6a55"
run_name = "lightgbm_experiment_5"
name = "lightgbm"
tags = {"version": "1.0", "type": "baseline", "model": name}

result = mlflow.register_model(
    model_uri=f"runs:/{run_id}/{run_name}",
    name=name,
    tags=tags,
    await_registration_for=150,
)

Successfully registered model 'lightgbm'.


RestException: RESOURCE_DOES_NOT_EXIST: Run '100d86e2471240dbb4b2199533bf6a55' not found